In [ ]:
# import the necessary libraries

import time
import gc
import os
import random

random_state = 42
os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin")
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
os.environ['PYTHONHASHSEED'] = str(random_state)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
log_dir = os.path.join(os.curdir, 'logs')

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from sklearn.model_selection import train_test_split
from skimage import io, util
from skimage.color import rgba2rgb
from tensorflow import keras

random.seed(random_state) # random python seed, but fixed
np.random.seed(random_state) # random numpy seed, but fixed
tf.random.set_seed(random_state) # random tf seed, but fixed

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# define the image path
img_train_folder_set1 = r'dataset\pictures\train\64x64\set-1'
img_train_folder_set2 = r'dataset\pictures\train\64x64\set-2'
img_test_folder = r'dataset\pictures\test\64x64'

# define plot properties for dark IDE
# font_size = 25
# font_color = 'white'
# plt.rc('axes', facecolor = '#1e1e1e')
# plt.rc('figure', facecolor = '#1e1e1e')
# plt.rc('axes', edgecolor = 'w')
# plt.rc('xtick', color = 'w')
# plt.rc('ytick', color = 'w')
# plt.rc('legend', facecolor = 'w')

# define plot properties for light IDE
font_size = 25
font_color = 'black'

In [ ]:
# auxiliary code to visualize the first 10 pictures of each category (SET-1)
plt.figure(figsize = (50, 50))
j = 0
counter_set1 = 0
counter_set2 = 0

for dir1 in os.listdir(img_train_folder_set1):
    for file in os.listdir(os.path.join(img_train_folder_set1, dir1)):
        image_path = os.path.join(img_train_folder_set1, dir1, file)  
        img = mpimg.imread(image_path)
        ax = plt.subplot(10, 10, j+1)
        ax.title.set_text(file)
        plt.axis('off')
        plt.title(dir1, fontsize = font_size, color = font_color)
        plt.imshow(img)
        j += 1
        counter_set1 += 1
        if counter_set1 == 10:
            break
    counter_set2 += 1
    counter_set1 = 0
    if counter_set2 == 7:
        break

In [ ]:
# auxiliary code to visualize the first 10 pictures of each category (SET-2)
plt.figure(figsize = (50, 50))
j = 0
counter_set3 = 0
counter_set4 = 0

for dir1 in os.listdir(img_train_folder_set2):
    for file in os.listdir(os.path.join(img_train_folder_set2, dir1)):
        image_path = os.path.join(img_train_folder_set2, dir1, file)  
        img = mpimg.imread(image_path)
        ax = plt.subplot(10, 10, j+1)
        ax.title.set_text(file)
        plt.axis('off')
        plt.title(dir1, fontsize = font_size, color = font_color)
        plt.imshow(img)
        j += 1
        counter_set3 += 1
        if counter_set3 == 10:
            break
    counter_set4 += 1
    counter_set3 = 0
    if counter_set4 == 7:
        break

In [ ]:
# function to gather pictures and generate a numpy array (COLOR)
def create_dataset(img_folder):
   
    img_data_array = []
    class_name = []
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path = os.path.join(img_folder, dir1, file)
            image = io.imread(image_path, as_gray = False)
            image = np.array(image)
            image = rgba2rgb(image)                      
            img_data_array.append(image)
            class_name.append(int(dir1))
    return img_data_array, class_name

In [ ]:
# function to gather pictures and generate a numpy array (GRAYSCALE)
def create_dataset(img_folder):
   
    img_data_array = []
    class_name = []
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path = os.path.join(img_folder, dir1, file)
            image = io.imread(image_path, as_gray = True)
            image = util.invert(image)
            image = np.array(image, dtype = 'float32')
            image /= 255
            img_data_array.append(image)
            class_name.append(int(dir1))
    return img_data_array, class_name

In [ ]:
# sets a image generator to inflate the dataset
generator = ImageDataGenerator(
    rotation_range = 90,
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    fill_mode='constant',
    cval=1)

In [ ]:
train_valid_size = 0.80
i = 0

# dataset import
X_array1, y_array1 = create_dataset(img_train_folder_set1)
X_array2, y_array2 = create_dataset(img_train_folder_set2)

X_trainSeq1 = np.array(X_array1)
y_trainSeq1 = np.array(y_array1)

X_trainSeq2 = np.array(X_array2)
y_trainSeq2 = np.array(y_array2)

## CNN grayscale only ##
#X_trainSeq1 = X_trainSeq1[..., np.newaxis]
#X_trainSeq2 = X_trainSeq2[..., np.newaxis]

X_gen = generator.flow(X_trainSeq1, y_trainSeq1, batch_size = 1820, seed = random_state)
X_trainGen1, y_trainGen1 = X_gen.next()
X_trainSeq1Gen = np.concatenate((X_trainSeq1, X_trainGen1), axis=0)
y_trainSeq1Gen = np.concatenate((y_trainSeq1, y_trainGen1), axis=0)
while i < 10:
    X_trainGen1, y_trainGen1 = X_gen.next()
    X_trainSeq1Gen = np.concatenate((X_trainSeq1Gen, X_trainGen1), axis=0)
    y_trainSeq1Gen = np.concatenate((y_trainSeq1Gen, y_trainGen1), axis=0)
    i += 1
i = 0

X_gen = generator.flow(X_trainSeq2, y_trainSeq2, batch_size = 329, seed = random_state)
X_trainGen2, y_trainGen2 = X_gen.next()
X_trainSeq2Gen = np.concatenate((X_trainSeq2, X_trainGen2), axis=0)
y_trainSeq2Gen = np.concatenate((y_trainSeq2, y_trainGen2), axis=0)
while i < 60:
    X_trainGen2, y_trainGen2 = X_gen.next()
    X_trainSeq2Gen = np.concatenate((X_trainSeq2Gen, X_trainGen2), axis=0)
    y_trainSeq2Gen = np.concatenate((y_trainSeq2Gen, y_trainGen2), axis=0)
    i += 1

counter_set1 = X_trainSeq1Gen.shape[0]
counter_set2 = X_trainSeq2Gen.shape[0]

X_totalTrainSet = np.concatenate((X_trainSeq1Gen, X_trainSeq2Gen), axis=0)
y_totalTrainSet = np.concatenate((y_trainSeq1Gen, y_trainSeq2Gen), axis=0)

X_train, X_valid, y_train, y_valid = train_test_split(X_totalTrainSet, y_totalTrainSet, train_size = train_valid_size, random_state = random_state, stratify = y_totalTrainSet)

del X_array1
del y_array1
del X_array2
del y_array2
del X_trainSeq1
del y_trainSeq1
del X_trainSeq2
del y_trainSeq2
del X_trainGen1
del y_trainGen1
del X_trainGen2
del y_trainGen2
del X_trainSeq1Gen
del y_trainSeq1Gen
del X_trainSeq2Gen
del y_trainSeq2Gen
del X_totalTrainSet
del y_totalTrainSet
del X_gen
gc.collect()

print('treinamento:         ', X_train.shape)
print('validação:           ', X_valid.shape)

In [ ]:
# enumerate the classes found
class_names = ['cpu', 'gpu', 'mobo', 'ram', 'hd', 'ssd-sata', 'ssd-m2']

In [ ]:
# train/validation/test dataset visualization (100 first items)
n_rows = 10
n_cols = 10
plt.figure(figsize = (n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index, :, :], cmap = 'binary')
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize = font_size / 2, color = font_color)
plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

In [ ]:
## CNN ##

epochs = 12

# build the neural network layers
keras.backend.clear_session() # clears the session
np.random.seed(random_state) # random tf seed, but fixed
tf.random.set_seed(random_state) # random tf seed, but fixed
if 'model' in globals():
        del model
        gc.collect()

model = keras.models.Sequential([
                keras.layers.Conv2D(filters = 32, kernel_size = 5, strides = 1, padding = 'same', activation = 'relu', input_shape = [64, 64, 3]),           
                keras.layers.MaxPool2D(pool_size = 2),
                keras.layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu'),              
                keras.layers.MaxPool2D(pool_size = 2),  
                keras.layers.Conv2D(filters = 96, kernel_size = 3, strides = 1, padding = 'same', activation = 'relu'),              
                keras.layers.MaxPool2D(pool_size = 2), 
                keras.layers.Flatten(),
                keras.layers.Dropout(0.5),
                keras.layers.Dense(512, activation = 'relu'),
                keras.layers.Dropout(0.25),
                keras.layers.Dense(256, activation = 'relu'),
                keras.layers.Dropout(0.25),
                keras.layers.Dense(64, activation = 'relu'),
                keras.layers.Dense(7, activation = 'softmax')
        ])        
model.summary()

In [ ]:
# prints the DNN/CNN structure
tf.keras.utils.plot_model(model, 'pc_hardware_model.png', show_shapes = True)

In [ ]:
temp = round((1 - train_valid_size) * 100, 2)

# sets NADAM as the optimizer (CNN)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'nadam', metrics = ['accuracy'])

run_dir1 = os.path.join(log_dir, 'cnn' + '-' + 'nadam' + '_' + 'eph' + str(epochs) + '_' + 'img' + str(X_train.shape[0] + X_valid.shape[0]) + '_' + str(64) + 'x' + str(64) + '_' + str(train_valid_size * 100) + '%' + '-' + str(temp) + '%' + '-' + '_' + time.strftime('run-%Y-%m-%d-%H-%M-%S'))
tensorboard1 = keras.callbacks.TensorBoard(run_dir1)

# trains the neural network
%time history = model.fit(X_train, y_train, epochs = epochs, validation_data = (X_valid, y_valid), callbacks = [tensorboard1])

In [ ]:
# prints the convergence graph
pd.DataFrame(history.history).plot(figsize=(12, 6))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
# loads Tensorboard
%load_ext tensorboard
%tensorboard --logdir=./logs --port=6006

In [ ]:
# function to gather pictures and generate a numpy array (COLOR)
def create_dataset2(img_folder):
   
    counter_set1 = 0
    img_data_array = []
    class_name = []
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path = os.path.join(img_folder, dir1, file)
            image = io.imread(image_path, as_gray = False)
            image = np.array(image, dtype = 'float32')  
            image /= 255          
            img_data_array.append(image)
            class_name.append(int(dir1))
            counter_set1 += 1
    return img_data_array, class_name, counter_set1

In [ ]:
# function to gather pictures and generate a numpy array (GRAYSCALE)
def create_dataset2(img_folder):
   
    counter_set1 = 0
    img_data_array = []
    class_name = []
   
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path = os.path.join(img_folder, dir1, file)
            image = io.imread(image_path, as_gray = True)
            image = util.invert(image)
            image = np.array(image, dtype = 'float32')
            image /= 255
            img_data_array.append(image)
            class_name.append(int(dir1))
    return img_data_array, class_name, counter_set1

In [ ]:
# dataset import
X_array2, y_array2, counter_set1 = create_dataset2(img_test_folder)

X_test = np.array(X_array2)
y_test = np.array(y_array2)

## CNN grayscale only ##
#X_test = X_test[..., np.newaxis]

from sklearn.utils import shuffle
X_test, y_test = shuffle(X_test, y_test, random_state = random_state)

print('testes finais:              ', X_test.shape)

In [ ]:
# prints the error data
print('erro de treino:   ', history.history['loss'][-1])
print('erro de validação:', history.history['val_loss'][-1])
print('erro de teste:    ', model.evaluate(X_test, y_test, verbose = 0))

In [ ]:
# probabilities for the first 100 test instances
X_new = X_test[:]
y_proba = model.predict(X_new)

In [ ]:
# predicted classes for the same 100 test instances
y_pred = np.argmax(model.predict(X_new), axis = -1)

In [ ]:
# test dataset evaluation view
n_rows = 12
n_cols = 12
plt.figure(figsize = (n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_new[index, :, :], cmap = 'binary')
        plt.axis('off')
        plt.title(class_names[y_pred[index]], fontsize = font_size / 2, color = font_color)
plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

In [ ]:
# show probability percentage
for image_index in range(30):
    plt.imshow(X_test[image_index])
    plt.show()
    for class_name, class_id in zip(class_names, y_proba[image_index]):
        print('{} - {:.2f}%'.format(class_name, class_id.round(2) * 100))
    print()

In [ ]:
# saves the CNN neural network
model.save('cnn' + '-' + 'nadam' + '_' + 'eph' + str(epochs) + '_' + 'img' + str(X_train.shape[0] + X_valid.shape[0]) + '_' + str(64) + 'x' + str(64) + '_' + str(train_valid_size * 100) + '%' + '-' + '%' + '-' + str(temp) + '%' + '-' + str(100 - (train_valid_size * 100) - temp) + '%' + '_' + time.strftime('run-%Y-%m-%d-%H-%M-%S') + '.h5')